<td>   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a></td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-notebooks/blob/main/project_configuration/prompt_response_projects.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-notebooks/tree/main/project_configuration/prompt_response_projects.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Prompt and response creation projects

This notebook will provide an example workflow of setting up a prompt and response type project with the Labelbox-Python SDK.

## Set up

In [ ]:
%pip install -q --upgrade "labelbox[data]"

In [ ]:
import labelbox as lb

## API key and client
Replace the value of `API_KEY` with a valid [API key]([ref:create-api-key](https://docs.labelbox.com/reference/create-api-key)) to connect to the Labelbox client.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Example: Create prompt and response projects and ontologies

The steps of creating prompt and response projects and corresponding ontologies using the Labelbox-Python SDK are similar to creating a regular project, and we will describe the slight differences for each scenario.

### Create a prompt and response ontology

You can create ontologies for prompt and response projects in the same way as other project ontologies using two methods: `client.create_ontology` and `client.create_ontology_from_feature_schemas`. You also need to include the respective `media_type`: `lb.MediaType.LLMPromptCreation` and `lb.MediaType.LLMPromptResponseCreation`. Additionally, you need to provide an `ontology_kind` parameter set to `lb.OntologyKind.ResponseCreation` that is only applicable for prompt and prompt response creation projects.

#### Option A: `client.create_ontology`

Typically, you create ontologies and generate the associated features simultaneously. Below is an example of creating an ontology for your prompt and response projects using supported classifications; for information on supported annotation types, see [prompt and response generation](https://docs.labelbox.com/docs/prompt-and-response-generation-editor).

Depending if you were creating a prompt, response, or prompt and response creation projects, you don't need certain classifications inside your ontologies. For information on supported annotation types, see [prompt and response generation](doc:prompt-and-response-generation-editor#supported-prompt-formats). In this notebook, we will create a prompt and response creation ontology. 

##### Prompt and response creation ontology

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[],
    classifications=[
        lb.PromptResponseClassification(
            class_type=lb.PromptResponseClassification.Type.PROMPT,
            name="prompt text",
            character_min=1,  # Minimum character count of prompt field (optional)
            character_max=
            20,  # Maximum character count of prompt field (optional)
        ),
        lb.PromptResponseClassification(
            class_type=lb.PromptResponseClassification.Type.RESPONSE_CHECKLIST,
            name="response checklist feature",
            options=[
                lb.Option(value="option 1", label="option 1"),
                lb.Option(value="option 2", label="option 2"),
            ],
        ),
        lb.PromptResponseClassification(
            class_type=lb.PromptResponseClassification.Type.RESPONSE_RADIO,
            name="response radio feature",
            options=[
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer"),
            ],
        ),
        lb.PromptResponseClassification(
            class_type=lb.PromptResponseClassification.Type.RESPONSE_TEXT,
            name="response text",
            character_min=
            1,  # Minimum character count of response text field (optional)
            character_max=
            20,  # Maximum character count of response text field (optional)
        ),
    ],
)

# Create ontology
ontology = client.create_ontology(
    "Prompt and response ontology",
    ontology_builder.asdict(),
    media_type=lb.MediaType.LLMPromptResponseCreation,
)

### Option B: `client.create_ontology_from_feature_schemas`
You can also create ontologies using feature schema IDs, which make your ontologies with existing features instead of generating new features. You can get these features by going to the _Schema_ tab inside Labelbox.

In [ ]:
# Uncomment the follwing code block for this option
# ontology = client.create_ontology_from_feature_schemas(
#     "LMC ontology",
#     feature_schema_ids=["<list of feature schema ids"],
#     media_type=lb.MediaType.Conversational,
#     ontology_kind=lb.OntologyKind.ModelEvaluation,
# )

### Create response creation projects

You can create response creation projects using `client.create_response_creation_project`, which uses the same parameters as `client.create_project` but provides better validation to ensure the project is set up correctly. Additionally, you need to import text data rows to be used as prompts.

In [ ]:
response_creation_project = client.create_response_creation_project(
    name="Demo response creation",
    description="<project_description>",  # optional
)

## Create prompt response and prompt creation projects

When creating a prompt response or prompt creation project using `client.create_prompt_response_generation_project()`, you do not need to create data rows because they are generated automatically. This method takes the same parameters as the traditional client.create_project but with a few specific additional parameters.

Parameters
The `client.create_prompt_response_generation_project` method requires the following parameters:

- `create_prompt_response_generation_project` parameters:

    - `name` (required): The name of your new project.

    - `description`: An optional description of your project.

    - `media_type` (required): The type of assets this project accepts. Can be either `lb.MediaType.LLMPromptCreation` or `MediaType.LLMPromptResponseCreation`, depending on the project type you are setting up.

    - `dataset_name`: The name of the dataset where the generated data rows will be located. Include this parameter only if you want to create a new dataset.

    - `dataset_id`: An optional dataset ID of an existing Labelbox dataset. Include this parameter if you want to append it to an existing dataset.

    - `data_row_count`: The number of data row assets that will be generated and used with your project.

In [ ]:
project = client.create_prompt_response_generation_project(
    name="Demo prompt response project",
    media_type=lb.MediaType.LLMPromptResponseCreation,
    dataset_name="Demo prompt response dataset",
    data_row_count=100,
)

# Setup project with ontology created above
project.connect_ontology(ontology)

## Exporting prompt response, prompt or response create project
Exporting from a prompt and response type project works the same as exporting from other projects. In this example, your export will be empty unless you create labels within the Labelbox platform. See prompt and response export for a sample export.

In [ ]:
# The return type of this method is an `ExportTask`, which is a wrapper of a`Task`
# Most of `Task` features are also present in `ExportTask`.

export_params = {
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "project_details": True,
    "label_details": True,
    "performance_details": True,
    "interpolated_frames": True,
}

# Note: Filters follow AND logic, so typically using one filter is sufficient.
filters = {
    "last_activity_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"],
    "label_created_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"],
    "workflow_status": "InReview",
    "batch_ids": ["batch_id_1", "batch_id_2"],
    "data_row_ids": ["data_row_id_1", "data_row_id_2"],
    "global_keys": ["global_key_1", "global_key_2"],
}

export_task = project.export(params=export_params, filters=filters)
export_task.wait_till_done()


# Return a JSON output string from the export task results/errors one by one:
def json_stream_handler(output: lb.BufferedJsonConverterOutput):
    print(output.json)


if export_task.has_errors():
    export_task.get_buffered_stream(stream_type=lb.StreamType.ERRORS).start(
        stream_handler=lambda error: print(error))

if export_task.has_result():
    export_json = export_task.get_buffered_stream(
        stream_type=lb.StreamType.RESULT).start(
            stream_handler=json_stream_handler)

print(
    "file size: ",
    export_task.get_total_file_size(stream_type=lb.StreamType.RESULT),
)
print(
    "line count: ",
    export_task.get_total_lines(stream_type=lb.StreamType.RESULT),
)

## Clean up

This section serves as an optional clean-up step to delete the Labelbox assets created within this guide. You will need to uncomment the delete methods shown.

In [ ]:
# project.delete()
# response_creation_project.delete()
# client.delete_unused_ontology(ontology.uid)
# dataset.delete()